# Clustering Crypto

In [91]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [51]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [52]:
# Create a DataFrame 
r = requests.get(url)
data = r.json()

In [53]:
list(data.keys())

['Response',
 'Message',
 'Data',
 'BaseImageUrl',
 'BaseLinkUrl',
 'RateLimit',
 'HasWarning',
 'Type']

In [54]:
# Use the 'Data' key from the json response, then transpose the DataFrame.
data = data['Data']

In [55]:
crypto_df_url = pd.DataFrame.from_dict(data)

In [60]:
crypto_df_url = crypto_df_url.T
crypto_df_url.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
## using the csv file as the url is not returning any column named "TotalCoinSupply"..  Tried maxSupply but it is giving differnt set of data 

In [156]:
# Alternatively, use the provided csv file:
crypto_df = pd.read_csv("../Data/crypto_data.csv", index_col='Unnamed: 0')
# Create a DataFrame
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [62]:
# Keep only necessary columns from url:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df_url.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'BlockNumber', 'NetHashesPerSecond',
       'BlockReward', 'BlockTime', 'AssetLaunchDate', 'MaxSupply',
       'MktCapPenalty', 'IsUsedInDefi', 'IsUsedInNft', 'PlatformType',
       'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty'],
      dtype='object')

In [63]:
crypto_df_url.drop(columns = [
    'Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
    'FullName', 'Description', 'AssetTokenStatus', 
    'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
    'BlockNumber', 'NetHashesPerSecond',
    'BlockReward', 'BlockTime', 'AssetLaunchDate', 
    'MktCapPenalty', 'IsUsedInDefi', 'IsUsedInNft', 'PlatformType',
    'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty'
], inplace = True)

In [64]:
crypto_df_url.head()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,0,0
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,0
404,404Coin,Scrypt,PoW/PoS,True,0,0
433,433 Token,N/A,N/A,False,NaN,NaN


In [105]:
## using the csv file as the url is not returning any column named "TotalCoinSupply"..  Tried maxSupply but it is giving differnt set of data 

In [157]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [158]:
# If'coerce', then invalid parsing will be set as NaN.
crypto_df['TotalCoinSupply'] = pd.to_numeric(crypto_df['TotalCoinSupply'], errors='coerce')

In [159]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
#crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [135]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [160]:
# Remove the "IsTrading" column
#crypto_df.drop('IsTrading', axis=1, inplace=True)
crypto_df.drop(columns = 'IsTrading', axis=1, inplace=True)

In [161]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      3
dtype: int64

In [162]:
coins_name = crypto_df.copy()

In [163]:
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [164]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)
#crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [166]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [167]:
crypto_df.drop('CoinName', axis=1, inplace=True)

In [168]:
# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummies.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummies)
print(crypto_scaled[0:1])

[[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
  -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
  -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
  -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
  -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
  -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
  -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
  -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.07523

In [170]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethash,PoW,1.076842e+08,0.000000e+00


In [171]:
crypto_scaled = crypto_scaled[~np.isnan(crypto_scaled).any(axis=1)]
np.isnan(crypto_scaled).sum()

0

### Reducing Dimensions Using PCA

In [180]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [181]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"],index=crypto_df.index 
)
pca_df.head()

,PC 1,PC 2,PC 3
42,-0.335201,0.960752,-0.482621
404,-0.318546,0.960758,-0.482749
1337,2.285004,1.606505,-0.528057
BTC,-0.147743,-1.295731,0.150332
ETH,-0.155607,-1.961910,0.300301


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [182]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [183]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pca_df)
# Predict clusters
predictions = model.predict(pca_df)
# Creating a new DataFrame including predicted clusters and cryptocurrencies features
pca_df["class"] = model.labels_
pca_df

,PC 1,PC 2,PC 3,class
42,-0.335201,0.960752,-0.482621,0
404,-0.318546,0.960758,-0.482749,0
1337,2.285004,1.606505,-0.528057,0
BTC,-0.147743,-1.295731,0.150332,3
ETH,-0.155607,-1.961910,0.300301,3
...,...,...,...,...
ZEPH,2.485121,1.025277,0.137967,0
GAP,-0.333248,0.960611,-0.482623,0
BDX,0.325352,-2.304752,0.304310,3
ZEN,-0.151523,-2.061915,0.284613,3


### Visualizing Results

#### 3D-Scatter with Clusters

In [184]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    pca_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [185]:
# Table with tradable cryptos

clustered_df = pd.merge(coins_name, pca_df, left_index=True, right_index=True)
clustered_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.335201,0.960752,-0.482621,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.318546,0.960758,-0.482749,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.285004,1.606505,-0.528057,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.147743,-1.295731,0.150332,3
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00,-0.155607,-1.961910,0.300301,3
...,...,...,...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09,2.485121,1.025277,0.137967,0
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08,-0.333248,0.960611,-0.482623,0
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09,0.325352,-2.304752,0.304310,3
ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07,-0.151523,-2.061915,0.284613,3


In [186]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [187]:
# Scale data to create the scatter plot
#copy dataframe
scaled_features = clustered_df.copy()
#selecct columns to scale
col_names = ['TotalCoinsMined', 'TotalCoinSupply']
features = scaled_features[col_names]
#scale with MinMaxScaler scaler
scaler = MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
#amend columns with scaled values
scaled_features[col_names] = features
scaled_features

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,42 Coin,Scrypt,PoW/PoS,0.005942,4.200000e-11,-0.335201,0.960752,-0.482621,0
404,404Coin,Scrypt,PoW/PoS,0.007002,5.320000e-04,-0.318546,0.960758,-0.482749,0
1337,EliteCoin,X13,PoW/PoS,0.035342,3.141593e-01,2.285004,1.606505,-0.528057,0
BTC,Bitcoin,SHA-256,PoW,0.005960,2.100000e-05,-0.147743,-1.295731,0.150332,3
ETH,Ethereum,Ethash,PoW,0.006050,0.000000e+00,-0.155607,-1.961910,0.300301,3
...,...,...,...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,0.007951,2.000000e-03,2.485121,1.025277,0.137967,0
GAP,Gapcoin,Scrypt,PoW/PoS,0.005957,2.500000e-04,-0.333248,0.960611,-0.482623,0
BDX,Beldex,CryptoNight,PoW,0.006927,1.400223e-03,0.325352,-2.304752,0.304310,3
ZEN,Horizen,Equihash,PoW,0.005950,2.100000e-05,-0.151523,-2.061915,0.284613,3


In [191]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_merged.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='class',   
    colormap="viridis", 
    hover_cols=["CoinName","Algorithm"]
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,class,CoinName,Algorithm)